In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.8 MB/s eta 0:00:00


In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.1 MB/s eta 0:00:00


In [5]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import List, Union
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import optuna

In [37]:
df = pd.read_csv('/content/drive/MyDrive/данные_рексис_проект/full_data.csv')
df

,product_id,price,user_id,Month,Day,is_weekend,hours_since_previous,cumulative_transaction_count,cumulative_price_mean,cumulative_price_median,...,758,759,760,761,762,763,764,765,766,767
0,1,1435.16,1,11,16,0,0.000000,6,529.261667,428.220,...,-0.318371,-0.242058,0.236013,-0.025708,0.528986,-0.190061,0.072266,-0.263037,-0.104120,-0.481117
1,2,1435.16,1,11,16,0,0.000000,7,658.675714,428.220,...,-0.318371,-0.242058,0.236013,-0.025708,0.528986,-0.190061,0.072266,-0.263037,-0.104120,-0.481117
2,3,1145.81,1,11,16,0,0.000000,8,719.567500,430.420,...,-0.318371,-0.242058,0.236013,-0.025708,0.528986,-0.190061,0.072266,-0.263037,-0.104120,-0.481117
3,4,138.87,1,11,16,0,0.000000,12,498.743333,381.925,...,-0.165869,-0.372402,0.218404,-0.056613,0.504462,-0.168178,0.050826,-0.208125,0.051956,-0.504573
4,5,30.53,1,11,16,0,0.000000,11,531.459091,428.220,...,-0.170266,-0.030963,0.320687,-0.035339,0.210574,0.070154,-0.023896,-0.133572,0.112684,-0.084872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216091,4971,694.42,9988,9,19,1,0.000000,1,694.420000,694.420,...,0.135994,-0.059023,0.177539,-0.021716,0.337237,-0.247520,0.034955,0.141408,-0.110053,-0.174521
216092,4971,694.42,9988,9,21,0,54.112222,2,694.420000,694.420,...,0.135994,-0.059023,0.177539,-0.021716,0.337237,-0.247520,0.034955,0.141408,-0.110053,-0.174521
216093,4971,694.42,9988,9,25,0,90.618889,3,694.420000,694.420,...,0.135994,-0.059023,0.177539,-0.021716,0.337237,-0.247520,0.034955,0.141408,-0.110053,-0.174521
216094,4971,694.42,9988,10,4,1,214.829444,4,694.420000,694.420,...,0.135994,-0.059023,0.177539,-0.021716,0.337237,-0.247520,0.034955,0.141408,-0.110053,-0.174521


The first step is to prepare our input data for the model

In [38]:
df = df.sort_values(by=['user_id', 'cumulative_transaction_count'])

In [39]:
df = df[['product_id','user_id']]
df = df.rename(columns={'product_id': 'item_id', 'user_id': 'clientid'})
df

,item_id,clientid
9,10,1
10,11,1
11,11,1
7,8,1
8,9,1
...,...,...
216090,9542,9988
216086,2156,9988
216089,36,9988
216095,1396,9988


In [44]:
num_unique_users = df['clientid'].nunique()

In [51]:
num_unique_users

9988

In [45]:
num_unique_items = df['item_id'].nunique()

In [52]:
num_unique_items

9657

In [50]:
Density = 216096 / (num_unique_users * num_unique_items)
print(Density * 100)

0.22404020581143474


In [9]:
df = (df.astype({'clientid': str, 'item_id': str})  # Convert specific columns to strings
      )[['clientid', 'item_id']]

In [10]:
def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0

    while start_index + window_size <= len(values):
        seq = values[start_index:start_index + window_size]
        sequences.append(seq)
        start_index += step_size

    # Padding for sequences with less than window_size
    if start_index < len(values):
        remaining = window_size - (len(values) - start_index)
        padded_seq =  [0] * remaining + values[start_index:]
        sequences.append(padded_seq)

    return sequences

In [11]:
df_group = df.groupby("clientid")

data = pd.DataFrame(
                     data={
                           "clientid": list(df_group.groups.keys()),
                           "item_id": list(df_group.item_id.apply(list)),
                           }
                    )
data.head()

,clientid,item_id
0,1,"[10, 11, 11, 8, 9, 1, 2, 3, 6, 7, 5, 4]"
1,10,"[124, 122, 123, 127, 125, 126, 125, 121, 128]"
2,100,"[778, 689, 164, 779, 780, 787, 781, 785, 784, ..."
3,1000,"[327, 328, 2460, 1591, 2662, 620, 522, 3313, 2..."
4,1001,"[1265, 3685, 1265, 3683, 1265, 3684, 3684, 368..."


In [12]:
sequence_length = 6
step_size = 1
data.item_id = data.item_id.apply(lambda ids: create_sequences(ids, sequence_length, step_size))
data.head()['item_id'].values

array([list([['10', '11', '11', '8', '9', '1'], ['11', '11', '8', '9', '1', '2'], ['11', '8', '9', '1', '2', '3'], ['8', '9', '1', '2', '3', '6'], ['9', '1', '2', '3', '6', '7'], ['1', '2', '3', '6', '7', '5'], ['2', '3', '6', '7', '5', '4'], [0, '3', '6', '7', '5', '4']]),
       list([['124', '122', '123', '127', '125', '126'], ['122', '123', '127', '125', '126', '125'], ['123', '127', '125', '126', '125', '121'], ['127', '125', '126', '125', '121', '128'], [0, '125', '126', '125', '121', '128']]),
       list([['778', '689', '164', '779', '780', '787'], ['689', '164', '779', '780', '787', '781'], ['164', '779', '780', '787', '781', '785'], ['779', '780', '787', '781', '785', '784'], ['780', '787', '781', '785', '784', '789'], ['787', '781', '785', '784', '789', '786'], ['781', '785', '784', '789', '786', '785'], ['785', '784', '789', '786', '785', '779'], ['784', '789', '786', '785', '779', '782'], ['789', '786', '785', '779', '782', '790'], ['786', '785', '779', '782', '790', '788'

In [13]:
data_transformed = data[["clientid", "item_id"]].explode(["item_id"], ignore_index=True)
data_transformed.head()

,clientid,item_id
0,1,"[10, 11, 11, 8, 9, 1]"
1,1,"[11, 11, 8, 9, 1, 2]"
2,1,"[11, 8, 9, 1, 2, 3]"
3,1,"[8, 9, 1, 2, 3, 6]"
4,1,"[9, 1, 2, 3, 6, 7]"


In [14]:
data_transformed['target'] = data_transformed['item_id'].apply(lambda x : np.array(x[-1], dtype=str))
data_transformed['item_id'] = data_transformed['item_id'].apply(lambda x : np.array(x[:-1], dtype=str))
data_transformed = data_transformed[['item_id', 'target']]

data_transformed.head()

,item_id,target
0,"[10, 11, 11, 8, 9]",1
1,"[11, 11, 8, 9, 1]",2
2,"[11, 8, 9, 1, 2]",3
3,"[8, 9, 1, 2, 3]",6
4,"[9, 1, 2, 3, 6]",7


In [15]:
# convert each variable to tensor and store in a dict then convert to tensorflow Dataset
data_transformed = {
                    'item_id': tf.constant(data_transformed["item_id"].to_list(), name='item_id'),
                    'target': tf.constant(data_transformed["target"], name='target')
                   }

tf_dict = tf.data.Dataset.from_tensor_slices(data_transformed)

# map rows to a dictionary
data = tf_dict.map(lambda x: {
                               "item_id": x["item_id"],
                               "target": x["target"]
                             }
                      )

items = tf.data.Dataset.from_tensor_slices({'item_id':df.item_id.unique()}).map(lambda x: x["item_id"])

data

<_MapDataset element_spec={'item_id': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'target': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [16]:
next(data.take(1).as_numpy_iterator())

{'item_id': array([b'10', b'11', b'11', b'8', b'9'], dtype=object),
 'target': b'1'}

In [17]:
def train_validation_test_split(map_tf_df: tf.data.Dataset,
                                data_len: int,
                                validation_share: float,
                                test_share: float) -> tuple:
    """
    Split a TensorFlow dataset into train, validation, and test sets.

    Args:
        map_tf_df (tf.data.Dataset): The dataset to split.
        data_len (int): The total length of the dataset.
        validation_share (float): The proportion of the data to allocate to the validation set (0.0 to 1.0).
        test_share (float): The proportion of the data to allocate to the test set (0.0 to 1.0).

    Returns:
        tuple: A tuple containing three datasets (train, validation, test).

    """
    # Shuffle the dataset
    shuffled = map_tf_df.shuffle(100_000, seed=1234, reshuffle_each_iteration=False)

    # Calculate sizes for each set
    train_size = int((1 - validation_share - test_share) * data_len)
    validation_size = int(validation_share * data_len)
    test_size = int(test_share * data_len)

    # Create train, validation, and test sets
    train = shuffled.take(train_size)
    validation = shuffled.skip(train_size).take(validation_size)
    test = shuffled.skip(train_size + validation_size).take(test_size)

    # Batch and cache the datasets
    cached_train = train.batch(4096).cache()
    cached_validation = validation.batch(4096).cache()
    cached_test = test.batch(4096).cache()

    return cached_train, cached_validation, cached_test

In [18]:
# Split the dataset to train validation and test
cached_train, cached_validation, cached_test = train_validation_test_split(data, len(data_transformed['target']), 0.15, 0.15)

Let's train our model using the prepared data

In [19]:
class GRU4REC(tfrs.Model):
    def __init__(self, embedding_dimension: int = 50, items: tf.data.Dataset = None,
                 add_layer: bool = False, dropout: float = 0.2, recurrent_dropout: float = 0.2,
                 batch_size: int = 256, learning_rate: float = 0.01, l2_lambda: float = 1e-6,
                 negative_samples: int = 10):
        super().__init__()

        self.items = items
        items_ids = self.items.batch(1_000)
        self.unique_items_ids = np.unique(np.concatenate(list(items_ids)))
        self.embedding_dimension = embedding_dimension
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.negative_samples = negative_samples

        # Query Model
        if add_layer:
            self._query_model = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=self.unique_items_ids, mask_token=None),
                tf.keras.layers.Embedding(len(self.unique_items_ids) + 1, self.embedding_dimension),

                tf.keras.layers.GRU(self.embedding_dimension, return_sequences=True,
                                    dropout=self.dropout, recurrent_dropout=self.recurrent_dropout),
                tf.keras.layers.GRU(self.embedding_dimension, dropout=self.dropout, recurrent_dropout=self.recurrent_dropout),
            ])
        else:
            self._query_model = tf.keras.Sequential([
                tf.keras.layers.StringLookup(
                    vocabulary=self.unique_items_ids, mask_token=None),
                tf.keras.layers.Embedding(len(self.unique_items_ids) + 1, self.embedding_dimension),

                tf.keras.layers.GRU(self.embedding_dimension, dropout=self.dropout, recurrent_dropout=self.recurrent_dropout),
            ])

        # Candidate Model
        self._candidate_model = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=self.unique_items_ids, mask_token=None),
            tf.keras.layers.Embedding(len(self.unique_items_ids) + 1, self.embedding_dimension)
        ])

        # Task and Metrics
        self._task = tfrs.tasks.Retrieval(
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=tfrs.metrics.FactorizedTopK(
                ks=[1, 5, 20, 50, 100], candidates=self.items.batch(128).map(self._candidate_model))
        )

    def compute_loss(self, features: dict, training: bool = False) -> Union[float, tf.Tensor]:
        watch_history = features["item_id"]
        watch_next_label = features["target"]

        query_embedding = self._query_model(watch_history)
        candidate_embedding = self._candidate_model(watch_next_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)


In [20]:
%%time
# Create and compile the model with specified parameters
model = GRU4REC(embedding_dimension=50,
                items=items,
                add_layer=False,
                dropout=0.2,
                recurrent_dropout=0.2,
                batch_size=256,
                learning_rate=0.01,
                l2_lambda=1e-6,
                negative_samples=10)

# Use custom retrieval task with TOP1 loss
model._task = CustomRetrieval(
    loss=None,  # Setting loss to None because it will be handled in compute_loss
    metrics=tfrs.metrics.FactorizedTopK(ks=[1, 5, 20, 50, 100], candidates=model.items.batch(128).map(model._candidate_model))
)

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01), loss=None)  # No need to specify loss during compilation

# Callbacks setting
es = EarlyStopping(monitor='val_factorized_top_k/top_20_categorical_accuracy', min_delta=0.01, patience=3, verbose=1)
mcp = ModelCheckpoint(filepath=r'gru_weights/weights{epoch:02d}.h5', monitor='val_factorized_top_k/top_20_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True)
rlr = ReduceLROnPlateau(monitor='val_factorized_top_k/top_20_categorical_accuracy', factor=0.5, patience=1, min_lr=1e-7, verbose=1)

# Training
history = model.fit(cached_train,
                    batch_size=256,
                    epochs=30,
                    validation_data=cached_validation,
                    callbacks=[es, mcp, rlr],
                    verbose=1)


Epoch 1/30
30/31 [============================>.] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_20_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 10.8378 - regularization_loss: 0.0000e+00 - total_loss: 10.8378
Epoch 1: val_factorized_top_k/top_20_categorical_accuracy improved from -inf to 0.00397, saving model to gru_weights/weights01.h5
31/31 [==============================] - 59s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_20_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 10.7529 - regularization_loss: 0.0000e+00 - total_loss: 10.7529 - val_factorized_top_k/to

Next, it is necessary to evaluate the operation of the constructed model. As mentioned above in the code, I tried models with different k-precision values. As a result, one best value will be added to the report

In [21]:
index = tfrs.layers.factorized_top_k.BruteForce(model._query_model, 20)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model._candidate_model)))
)

In [22]:
np.expand_dims(next(cached_test.as_numpy_iterator())['item_id'][42], axis=0)

array([[b'389', b'1795', b'8005', b'9349', b'7986']], dtype=object)

In [23]:
# make prediction for each test observation. top 20
test = np.array(list(cached_test.as_numpy_iterator()))

item_id_values = np.concatenate([i['item_id'] for i in test])
target_values = np.concatenate([i['target'] for i in test])

predictions = [index(np.expand_dims(item_id_values[i], axis=0))[1].numpy()[0].tolist() for i in tqdm(range(len(target_values)))]

100%|██████████| 26421/26421 [24:53<00:00, 17.70it/s]


In [24]:
_, rec = index(np.expand_dims(next(cached_test.as_numpy_iterator())['item_id'][42], axis=0))
rec

<tf.Tensor: shape=(1, 20), dtype=string, numpy=
array([[b'315', b'3309', b'1137', b'492', b'4813', b'1043', b'450',
        b'1673', b'1920', b'1885', b'4376', b'2910', b'1134', b'399',
        b'3610', b'7315', b'4354', b'4492', b'3479', b'754']],
      dtype=object)>

In [25]:
model.evaluate(cached_train)

31/31 [==============================] - 141s 5s/step - factorized_top_k/top_1_categorical_accuracy: 2.0276e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_20_categorical_accuracy: 0.0118 - factorized_top_k/top_50_categorical_accuracy: 0.0223 - factorized_top_k/top_100_categorical_accuracy: 0.0366 - loss: 10.8018 - regularization_loss: 0.0000e+00 - total_loss: 10.8018


[0.00020275749557185918,
 0.003033252200111747,
 0.011784265749156475,
 0.02227899432182312,
 0.03659367561340332,
 10.163023948669434,
 0,
 10.163023948669434]

In [32]:
model.evaluate(cached_validation)

7/7 [==============================] - 39s 6s/step - factorized_top_k/top_1_categorical_accuracy: 7.5697e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0025 - factorized_top_k/top_20_categorical_accuracy: 0.0111 - factorized_top_k/top_50_categorical_accuracy: 0.0208 - factorized_top_k/top_100_categorical_accuracy: 0.0350 - loss: 10.8903 - regularization_loss: 0.0000e+00 - total_loss: 10.8903


[7.569736044388264e-05,
 0.002460164250805974,
 0.011127511970698833,
 0.020816775038838387,
 0.03501002863049507,
 10.569043159484863,
 0,
 10.569043159484863]

In [33]:
model.evaluate(cached_test)

7/7 [==============================] - 31s 4s/step - factorized_top_k/top_1_categorical_accuracy: 2.2709e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0036 - factorized_top_k/top_20_categorical_accuracy: 0.0123 - factorized_top_k/top_50_categorical_accuracy: 0.0225 - factorized_top_k/top_100_categorical_accuracy: 0.0382 - loss: 10.9100 - regularization_loss: 0.0000e+00 - total_loss: 10.9100


[0.00022709208133164793,
 0.003633473301306367,
 0.012262973003089428,
 0.022482115775346756,
 0.03818931803107262,
 10.635590553283691,
 0,
 10.635590553283691]

In [28]:
test = np.array(list(cached_test.as_numpy_iterator()))

item_id_values = np.concatenate([i['item_id'] for i in test])
target_values = np.concatenate([i['target'] for i in test])

predictions = [index(np.expand_dims(item_id_values[i], axis=0))[1].numpy()[0].tolist() for i in tqdm(range(len(target_values)))]

100%|██████████| 26421/26421 [23:51<00:00, 18.46it/s]


Functions of standard metrics for evaluating the quality of the model:

In [34]:
def recall_at_k(predictions: np.array, targets: np.array, k: int) -> float:
    """
    Calculate Recall@k.

    Parameters:
    - predictions: 2D NumPy array, where each row represents the ranked list of relevance scores.
                   The first element in each row is considered the most relevant.
    - targets: 2D NumPy array, where each row represents the ground truth relevance scores.
               It should have the same structure as the predictions.
    - k: Integer, the cutoff rank for evaluation.

    Returns:
    - recall: Float, the Recall@k.
    """
    top_k_predictions = predictions[:, :k]
    recall = np.mean([np.isin(target, top_k_predictions[i, :]) for i, target in enumerate(targets)])
    return recall


def mrr_at_k(predictions: np.array, targets: np.array, k: int) -> float:
    """
    Calculate MRR@k.

    Parameters:
    - predictions: 2D NumPy array, where each row represents the ranked list of relevance scores.
                   The first element in each row is considered the most relevant.
    - targets: 2D NumPy array, where each row represents the ground truth relevance scores.
               It should have the same structure as the predictions.
    - k: Integer, the cutoff rank for evaluation.

    Returns:
    - mrr: Float, the MRR@k.
    """
    rr = []
    for pred, target in zip(predictions[:, :k], targets):
        if target in pred:
            rr.append((1/(np.where(pred == target)[0][0]+1)))
        else:
            rr.append(0)
    return np.mean(rr)


def ndcg_at_k(predicted_ranking, actual_ranking, k):
    """
    Calculate Normalized Discounted Cumulative Gain at k.

    Parameters:
    - actual_ranking: List of true rankings (ground truth).
    - predicted_ranking: List of predicted rankings.
    - k: The position to consider in the rankings.

    Returns:
    - NDCG@k score.
    """
    if len(actual_ranking) != len(predicted_ranking):
        raise ValueError("Length of actual and predicted rankings must be the same.")

    dcg = 0.0
    idcg = 0.0

    for i in range(len(actual_ranking)):
        actual_rank = actual_ranking[i]
        predicted_rank = predicted_ranking[i][:k]

        # Calculate DCG (Discounted Cumulative Gain)
        gain = 2 ** 1 - 1 if actual_rank in predicted_rank else 0
        dcg += gain / np.log2(i + 2)

        # Calculate ideal DCG for normalization
        gain_idcg = 2 ** 1 - 1
        idcg += gain_idcg / np.log2(i + 2)

    # Avoid division by zero
    if idcg == 0:
        return 0.0

    # Calculate NDCG@k
    ndcg = dcg / idcg
    return ndcg

In [36]:
for func in [recall_at_k, mrr_at_k, ndcg_at_k]:
    print(f"\n{func.__name__}")
    print("="*40)
    for k in [1, 5, 20, 50, 100]:
        print(f"{func.__name__} at {k}: {func(np.array(predictions), target_values, k=k)}")


recall_at_k
recall_at_k at 1: 0.00037848680973468076
recall_at_k at 5: 0.0038984141402672117
recall_at_k at 20: 0.012338669997350593
recall_at_k at 50: 0.012338669997350593
recall_at_k at 100: 0.012338669997350593

mrr_at_k
mrr_at_k at 1: 0.00037848680973468076
mrr_at_k at 5: 0.00144014231104046
mrr_at_k at 20: 0.002227287324856947
mrr_at_k at 50: 0.002227287324856947
mrr_at_k at 100: 0.002227287324856947

ndcg_at_k
ndcg_at_k at 1: 0.0003698234471600101
ndcg_at_k at 5: 0.0038241648832024766
ndcg_at_k at 20: 0.012333054501996375
ndcg_at_k at 50: 0.012333054501996375
ndcg_at_k at 100: 0.012333054501996375
